In [1]:
!pip install transformers
!pip install datasets
!pip install huggingface_hub

     |████████████████████████████████| 3.8 MB 2.7 MB/s 
     |████████████████████████████████| 895 kB 35.6 MB/s 
     |████████████████████████████████| 6.5 MB 53.3 MB/s 
     |████████████████████████████████| 67 kB 3.8 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 59.8 MB/s 
     |████████████████████████████████| 134 kB 64.3 MB/s 
     |████████████████████████████████| 212 kB 55.9 MB/s 
     |████████████████████████████████| 127 kB 49.9 MB/s 
     |████████████████████████████████| 144 kB 59.3 MB/s 
     |████████████████████████████████| 271 kB 56.9 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels = 3)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset, DatasetDict
tweets = load_dataset('chiarab/final-train', use_auth_token=True)

Using custom data configuration chiarab--final-train-bb31ffa1a33f0c4c


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/441k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/chiarab--final-train-bb31ffa1a33f0c4c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
print(tweets)

DatasetDict({
    train: Dataset({
        features: ['0', '1'],
        num_rows: 3170
    })
})


In [5]:
# Prepare the small dataset - this tokenizes the dataset in batches of 16 examples.
finetune_small = DatasetDict(
    train=tweets['train'].shuffle(seed=1111).select(range(128)),
    val=tweets['train'].shuffle(seed=1111).select(range(128, 160)),
    test=tweets['train'].shuffle(seed=1111).select(range(160, 260))
)

small_tokenized_dataset = finetune_small.map(
    lambda example: tokenizer(example['0'], padding=True),
    batched=True,
    batch_size=16
)

small_tokenized_dataset = small_tokenized_dataset.remove_columns(["0"])
small_tokenized_dataset = small_tokenized_dataset.rename_column("1", "labels")
small_tokenized_dataset.set_format("torch")

print(small_tokenized_dataset)

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/chiarab--final-train-bb31ffa1a33f0c4c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-646aeb2a9d43e2d6.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/chiarab--final-train-bb31ffa1a33f0c4c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-646aeb2a9d43e2d6.arrow


  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 128
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 32
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})


In [6]:
# Prepare the small dataset - this tokenizes the dataset in batches of 16 examples.
finetune = DatasetDict(
    train=tweets['train'].shuffle(seed=1111).select(range(2219)),
    val=tweets['train'].shuffle(seed=1111).select(range(2219, 2695)),
    test=tweets['train'].shuffle(seed=1111).select(range(2695, 3170))
)

tokenized_dataset = finetune.map(
    lambda example: tokenizer(example['0'], padding=True),
    batched=True,
    batch_size=16
)

tokenized_dataset = tokenized_dataset.remove_columns(["0"])
tokenized_dataset = tokenized_dataset.rename_column("1", "labels")
tokenized_dataset.set_format("torch")

print(tokenized_dataset)

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/chiarab--final-train-bb31ffa1a33f0c4c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-646aeb2a9d43e2d6.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/chiarab--final-train-bb31ffa1a33f0c4c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-646aeb2a9d43e2d6.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/chiarab--final-train-bb31ffa1a33f0c4c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-646aeb2a9d43e2d6.arrow


  0%|          | 0/139 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2219
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 476
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 475
    })
})


In [7]:
from transformers import TrainingArguments, Trainer
import numpy as np

arguments = TrainingArguments(
    output_dir="sample_hf_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224
)


def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print("predictions")
    print(predictions)
    print("labels")
    print(labels)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [8]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2219
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 417


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.908094,0.600840
2,No log,0.899497,0.623950
3,No log,0.927092,0.630252


***** Running Evaluation *****
  Num examples = 476
  Batch size = 16


predictions
[2 1 0 1 1 2 0 0 1 0 0 1 1 1 1 1 1 1 0 1 2 1 1 1 1 1 0 0 1 1 2 1 1 2 1 0 1
 2 1 1 1 1 2 2 2 0 2 1 1 1 1 0 1 1 2 2 1 0 1 1 1 2 1 1 1 1 1 1 2 1 2 1 1 1
 0 0 0 2 1 0 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 2 1 2 1 2 1 1 0 1 1
 1 1 1 1 0 0 1 1 1 1 1 1 0 1 2 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1
 2 1 2 1 1 1 1 1 1 1 1 2 1 1 2 0 1 2 1 2 1 1 0 2 0 1 1 1 1 1 1 1 2 0 1 2 2
 1 0 1 1 2 0 0 2 1 1 2 2 2 1 1 1 1 0 1 2 1 2 1 1 1 1 1 1 1 0 2 1 1 2 1 1 1
 1 1 2 1 1 1 1 1 2 1 1 2 0 2 2 1 1 2 0 1 1 2 1 1 1 2 1 1 1 1 1 2 0 2 1 1 1
 1 1 0 2 1 2 1 2 0 0 1 1 1 1 1 1 1 0 2 2 1 2 1 2 1 0 1 0 1 2 0 2 1 2 0 1 1
 1 1 1 1 2 1 2 1 2 2 1 1 1 1 1 1 1 1 0 1 0 2 1 1 1 1 1 1 1 1 1 1 1 0 2 1 1
 2 1 1 0 2 1 1 1 1 0 1 1 1 1 2 1 1 0 1 2 1 1 0 2 0 1 1 0 2 2 2 1 1 2 1 1 1
 1 1 1 0 1 1 1 0 1 2 1 1 1 1 1 1 1 2 0 2 1 0 1 1 1 0 1 1 1 1 1 1 0 2 1 2 1
 1 0 1 1 2 1 1 1 1 1 1 1 0 1 2 1 2 1 1 0 1 2 1 1 0 2 1 1 1 1 1 1 2 0 1 1 2
 1 0 1 1 2 0 2 1 1 1 1 0 1 1 1 1 1 0 2 1 0 2 0 2 1 1 1 1 1 0 2 1]
labels
[2 2 0 0 1 2 0 

Saving model checkpoint to sample_hf_trainer/checkpoint-139
Configuration saved in sample_hf_trainer/checkpoint-139/config.json
Model weights saved in sample_hf_trainer/checkpoint-139/pytorch_model.bin
tokenizer config file saved in sample_hf_trainer/checkpoint-139/tokenizer_config.json
Special tokens file saved in sample_hf_trainer/checkpoint-139/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 476
  Batch size = 16


predictions
[2 1 0 1 1 2 0 0 1 0 0 1 1 1 1 1 1 1 0 1 2 1 1 1 0 1 0 0 1 1 2 1 1 2 1 0 1
 2 1 1 1 1 2 2 2 0 2 1 1 1 1 0 1 1 2 2 1 1 1 1 1 2 1 1 1 1 1 1 2 1 2 1 1 1
 0 0 2 2 1 0 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 2 1 2 1 2 1 0 0 0 1
 1 0 1 1 0 0 1 1 2 1 1 1 0 2 2 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1
 2 1 2 1 1 1 1 1 1 1 1 2 1 1 2 0 1 2 1 2 1 1 0 2 0 1 1 1 0 1 1 1 2 0 1 2 0
 1 0 1 1 2 0 0 2 0 2 2 2 2 1 1 1 1 0 1 2 1 2 1 1 1 1 1 1 1 0 2 1 0 2 1 0 1
 1 1 2 1 1 1 1 1 2 1 1 2 0 2 2 1 2 2 0 1 1 2 1 1 1 0 1 1 1 1 1 2 0 2 0 1 1
 2 1 0 2 1 2 1 2 0 0 1 1 1 1 1 1 1 0 2 2 2 2 1 2 1 0 1 0 0 2 0 2 1 2 0 1 1
 1 1 1 1 2 1 2 1 2 2 1 1 1 1 1 1 1 1 0 1 0 2 1 1 1 1 1 1 1 1 1 1 1 0 2 1 1
 2 1 1 0 2 1 2 1 1 0 1 1 1 1 2 1 1 0 1 2 1 1 0 2 0 1 1 2 2 2 2 1 1 2 1 1 1
 1 1 0 0 1 1 1 0 1 2 1 1 1 0 1 1 1 2 0 2 1 0 1 1 1 0 1 1 1 1 1 1 0 2 1 2 1
 0 0 1 1 0 1 1 1 1 1 2 1 0 1 2 1 2 1 1 0 1 2 1 0 0 2 1 1 1 1 1 1 2 0 1 1 0
 1 0 1 1 1 0 2 2 1 1 1 0 1 1 1 1 1 0 2 1 0 2 0 2 1 1 2 1 0 0 2 1]
labels
[2 2 0 0 1 2 0 

Saving model checkpoint to sample_hf_trainer/checkpoint-278
Configuration saved in sample_hf_trainer/checkpoint-278/config.json
Model weights saved in sample_hf_trainer/checkpoint-278/pytorch_model.bin
tokenizer config file saved in sample_hf_trainer/checkpoint-278/tokenizer_config.json
Special tokens file saved in sample_hf_trainer/checkpoint-278/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 476
  Batch size = 16


predictions
[2 1 0 0 1 2 0 0 0 0 0 1 1 1 1 0 1 1 0 1 2 1 1 1 0 1 0 0 1 1 2 1 1 2 1 0 0
 2 1 1 1 1 2 2 2 0 2 0 1 1 1 0 1 1 2 2 1 0 1 1 1 2 1 1 1 2 1 1 2 1 2 1 1 1
 0 0 0 2 1 0 1 1 1 2 1 1 0 1 1 1 1 0 1 1 2 2 1 2 1 1 1 2 1 2 1 2 0 0 0 0 1
 1 0 1 1 0 0 1 0 2 1 1 1 0 2 2 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1
 2 1 2 1 1 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 0 2 0 1 1 1 0 0 1 1 2 0 1 2 0
 1 0 1 1 2 0 0 2 0 2 2 2 2 1 1 1 1 0 1 2 1 2 1 1 1 1 1 1 1 0 2 1 0 2 1 0 0
 1 1 2 1 1 1 1 0 2 1 1 2 0 2 2 1 1 2 0 1 1 2 0 1 1 0 1 1 1 1 1 2 0 2 0 0 1
 2 1 0 2 1 2 1 2 0 0 1 1 1 1 1 1 1 0 2 2 2 2 1 2 1 0 1 0 0 2 0 2 1 2 0 1 0
 1 1 1 1 2 1 2 0 2 2 1 1 1 1 1 1 1 1 0 2 0 2 1 1 1 1 0 1 1 1 1 1 1 0 2 1 1
 2 1 1 0 2 1 2 0 1 0 1 1 1 1 2 1 1 0 1 2 1 1 0 2 0 1 1 2 2 2 2 1 1 0 1 1 1
 0 1 0 0 1 1 1 0 1 2 1 1 1 0 1 1 2 2 0 2 1 0 1 1 0 0 1 1 1 1 1 1 0 2 1 2 1
 0 0 1 1 0 1 1 1 1 1 2 1 0 1 2 1 2 1 1 0 1 2 1 0 0 2 1 1 1 1 1 1 2 0 1 1 0
 1 0 1 1 1 0 2 2 1 1 1 0 1 0 1 1 1 0 2 2 0 2 0 2 1 1 2 1 0 0 2 1]
labels
[2 2 0 0 1 2 0 

Saving model checkpoint to sample_hf_trainer/checkpoint-417
Configuration saved in sample_hf_trainer/checkpoint-417/config.json
Model weights saved in sample_hf_trainer/checkpoint-417/pytorch_model.bin
tokenizer config file saved in sample_hf_trainer/checkpoint-417/tokenizer_config.json
Special tokens file saved in sample_hf_trainer/checkpoint-417/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from sample_hf_trainer/checkpoint-278 (score: 0.899497389793396).


TrainOutput(global_step=417, training_loss=0.7488788266261991, metrics={'train_runtime': 6700.0125, 'train_samples_per_second': 0.994, 'train_steps_per_second': 0.062, 'total_flos': 521630059046670.0, 'train_loss': 0.7488788266261991, 'epoch': 3.0})

In [ ]:
results = trainer.predict(tokenized_dataset['test'])

***** Running Prediction *****
  Num examples = 475
  Batch size = 16


In [22]:
results

PredictionOutput(predictions=array([[-1.432071  ,  1.9613048 , -0.5784473 ],
       [ 0.9289329 ,  1.2532382 , -2.0851223 ],
       [-0.17961988,  0.7034045 , -0.5075814 ],
       [-2.2624936 ,  3.0801406 , -0.92738754],
       [ 4.2000527 , -1.8458118 , -2.14575   ],
       [-3.53072   ,  3.111443  ,  0.16128136],
       [ 3.6891363 , -1.4360029 , -2.0565152 ],
       [ 0.43140554,  0.23942375, -0.34057432],
       [-0.42840046,  1.2962024 , -0.82384396],
       [-0.53994536,  1.5092466 , -0.95394796],
       [-0.82545435,  2.2222397 , -1.4489131 ],
       [ 3.6393762 , -1.7233101 , -1.7487816 ],
       [-1.4533405 , -0.22341079,  2.106429  ],
       [-1.9847769 , -1.6038997 ,  4.446768  ],
       [-0.25979525,  1.3975145 , -1.1242179 ],
       [-1.3891709 ,  3.160619  , -2.0393176 ],
       [ 2.0625806 , -0.97624886, -0.9126157 ],
       [-1.6329064 ,  2.6324954 , -1.1124582 ],
       [-2.5303783 ,  3.377993  , -1.0941377 ],
       [ 1.040904  ,  1.084909  , -2.1968656 ],
       [ 0.

In [ ]:
correct = 0
wrong = 0

# Initialize the regular tokenizer
tokenizer2 = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
# Initialize the regular model
model2 = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment", num_labels = 3)

model2.eval()
for tweet in finetune_test:
  inputs = tweet['OriginalTweet']
  tokenized_inputs = tokenizer2(inputs, return_tensors="pt")
  outputs = model2(**tokenized_inputs)
  labels = [0, 1, 2]
  prediction = torch.argmax(outputs.logits)
  if prediction == tweet['Sentiment']:
    correct += 1
  else:
    wrong += 1

print("Regular Accuracy:")
print(correct / (correct + wrong))
print(correct + wrong)

Regular Accuracy:
0.5202312138728323
173


In [ ]:
model.eval()
neg = 0
neu = 0
pos = 0

for tweet in tweets_vaccine:
  inputs = tweet['OriginalTweet']
  tokenized_inputs = tokenizer(inputs, return_tensors="pt")
  outputs = model(**tokenized_inputs)
  labels = [0, 1, 2]
  prediction = torch.argmax(outputs.logits)
  if prediction == 0:
    neg += 1
  elif prediction == 1:
    neu += 1
  else:
    pos += 1

print(neg)
print(neu)
print(pos)

print(prediction)

0
553
0
tensor(1)
